# 非同期処理

In [1]:
import os
os.environ["OPENAI_API_KEY"] = "sk-"

In [2]:
import warnings
warnings.simplefilter('ignore')

In [ ]:
import asyncio
import logging
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

# 以下、jupyterで実行する場合
# !pip install nest-asyncio
import nest_asyncio
nest_asyncio.apply()

In [119]:
# GPT-3からの応答を取得する関数
def get_gpt_response(qa,question: str) -> str:
    try:
        reply = qa.run(question)
        return reply
    
    except Exception as e:
        # エラーロギング
        logging.error(f"エラー: {str(e)}")
        raise ValueError(f"エラーが発生しました。{str(e)}") 

<font size="4">`df`にQuestionが格納されているとする

In [120]:
def async_gpt_responses(df,qa):
    try:
        # 新しい非同期ループを作成
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        
        with ThreadPoolExecutor() as executor:
            tasks = []
            for i in range(len(df)):
                question = df.iloc[i]["question"]
                task = loop.run_in_executor(executor, get_gpt_response, qa, question)
                tasks.append(task)
                        
            responses = loop.run_until_complete(asyncio.gather(*tasks))
        
    except Exception as e:
        # エラーロギング
        logging.error(f"エラー: {str(e)}")
        raise ValueError(f"エラーが発生しました。{str(e)}")
    finally:
        # イベントループのクローズ
        loop.close()

    return responses

In [310]:
%%time
# 非同期で応答を取得
responses = async_gpt_responses(df_qa_50,qa)

CPU times: user 910 ms, sys: 157 ms, total: 1.07 s
Wall time: 24.9 s


In [311]:
df.loc[:,"Answer"] = responses